### Scraping HTML pages from indeed.com 
This program takes a list of job titles and a list of cities, it performs a job search on indeed.com, and it saves the HTML page of each job posting found. All the job posting .html files are saved in one folder and  the meta data associated with each posting is saved in an SQL database stored in a .sqlite file.

##### Steps that the program is performing:
1. Loop through all the job titles in the list.
2. For every job title, loop through all the cities in the list.
3. For each job and city, perform a search on indeed.com.
4. Go though all the results pages, extract the ID of each posting and perform the following 2 tasks:
    - Store the meta data about the posting in the job_search SQL table. The <b>job_search table</b> stores the following information:
<table>
  <tr>
    <th style="text-align:center"> job_search SQL Table </th>
  </tr>
  <tr>
    <td>
      <table>
        <tr>
          <th style="text-align:center"> id </th>
          <th style="text-align:center"> job_seach_word </th>
          <th style="text-align:center"> city </th>
          <th style="text-align:center"> province </th>
          <th style="text-align:center"> page </th>
          <th style="text-align:center"> jk_id </th>
          <th style="text-align:center"> url</th>
        </tr>
        <tr>
          <td style="text-align:center"> auto-increment ID </td>
          <td style="text-align:center"> the word used to perform the search on indeed.com </td>
          <td style="text-align:center"> the city name used to perform the search on indeed.com </td>
          <td style="text-align:center"> the province name used to perform the search on indeed.com</td>
          <td style="text-align:center"> the number of the results page that the job posting was found on</td>
          <td style="text-align:center"> the job ID associated with each posting given by indeed.com</td>
          <td style="text-align:center"> the URL of the individual job posting.</td>
        </tr>
      </table>
    </td>
  </tr>
</table>
    - Check to see if the jk_id for each listing is present in the listings SQL table. If it is not, store the jk_id and the url of the posting in the job_urls list. 
5. Go though all the jk_ids and the urls in job_url list. These job listings have not been yet scraped. For each job posting perform the following task:
    - Access the URL of the job posting.
    - Save the HTML code in a .html file locally on the computer. 
    - Get the jk_id and url for up to 5 recommended jobs showed on this listing. Add a new row to the <b>listing table</b> that includes the following information.
<table>
  <tr>
    <th style="text-align:center"> listing SQL Table </th>
  </tr>
  <tr>
    <td>
      <table>
        <tr>
          <th style="text-align:center"> id </th>
          <th style="text-align:center"> jk_id </th>
          <th style="text-align:center"> title </th>
          <th style="text-align:center"> recommendation 1_jk </th>
          <th style="text-align:center"> recommendation 2_jk </th>
          <th style="text-align:center"> recommendation 3_jk </th>
          <th style="text-align:center"> recommendation 4_jk </th>
          <th style="text-align:center"> recommendation 5_jk </th>
        </tr>
        <tr>
          <td style="text-align:center"> auto-increment ID </td>
          <td style="text-align:center"> the job_id of this specific posting found on indeed.com </td>
          <td style="text-align:center"> the title of the job posting </td>
          <td style="text-align:center"> the jk_id of the first recommended job found on this listing.</td>
          <td style="text-align:center"> the jk_id of the second recommended job found on this listing.</td>
          <td style="text-align:center"> the jk_id of the third recommended job found on this listing.</td>
          <td style="text-align:center"> the jk_id of the forth recommended job found on this listing.</td>
          <td style="text-align:center"> the jk_id of the fifth recommended job found on this listing.</td>
        </tr>
      </table>
    </td>
  </tr>
</table>
    - For each recommended job found of the job posting, perform the following two tasks:
        - Add a new line to the <b>recommendation table</b> that includes the following information:
<table>
  <tr>
    <th style="text-align:center"> recommendation SQL Table </th>
  </tr>
  <tr>
    <td>
      <table>
        <tr>
          <th style="text-align:center"> id </th>
          <th style="text-align:center"> jk_id </th>
          <th style="text-align:center"> url </th>
          <th style="text-align:center"> parent_jk </th>
        </tr>
        <tr>
          <td style="text-align:center"> auto-increment ID </td>
          <td style="text-align:center"> the jk_id of the recommendation posting </td>
          <td style="text-align:center"> the url of the recommentation posting </td>
          <td style="text-align:center"> the jk_id of the job listing where the recommended job was found</td>
        </tr>
      </table>
    </td>
  </tr>
</table>
        - Check to see if the recommended job is in the listing table. If it is not, make a request and access the url of the recommended job posting, save the HTML code in a .html file on the computer and add a new line to the listings table. 


Assumptions:
    - every job has a unique jk_id it found in the href of <a> tag on the webpage. 
    - the jk_id is found in the URL of the job posting
    - for every job posting, up to 5 recommended jobs will scraped 

In [30]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import logging
import datetime as dt
import pandas as pd
import os

In [2]:
from sqlalchemy import create_engine, inspect
from sqlalchemy import MetaData, Table, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

## Set up database in sqlite file

In [3]:
path = "indeed_scraped_db"
engine = create_engine(f"sqlite:///{path}")
conn = engine.connect()
session = Session(bind=engine)

In [4]:
Base = declarative_base()

In [5]:
class Job_Search(Base):
    __tablename__ = "job_seach"
    id = Column(Integer, primary_key = True)
    job_search_word = Column(String(255))
    city = Column(String(255))
    province = Column(String(255))
    page = Column(Integer)
    jk_id = Column(String(255))
    url = Column(String(255)) 

In [6]:
class Listing(Base):
    __tablename__ = "listing"
    id = Column(Integer, primary_key = True)
    jk_id = Column(String(255))
    title = Column(String(255))
    recommendation1_jk = Column(String(255))
    recommendation2_jk = Column(String(255))
    recommendation3_jk = Column(String(255))
    recommendation4_jk = Column(String(255))
    recommendation5_jk = Column(String(255))

In [7]:
class Recommendation(Base):
    __tablename__ = "recommendation"
    id = Column(Integer, primary_key = True)
    jk_id = Column(String(255))
    rec_url = Column(String(255))
    parent_jk = Column(String(255))

In [8]:
Base.metadata.create_all(engine) 

In [9]:
today = dt.datetime.now().strftime("%d-%b-%Y")
f = f"Logging {today}.txt"
logging.basicConfig(filename = f,level = logging.DEBUG)

###### Funcation: get_jobURL_and_jk(href)
Input: string containing the href value found in the < a >  tag that contains the link to a single jobpost.
- The jk_id is found in the href. 
- An URL can be built from the href.

Output: Returns the 16-digit jk_id and the URL of a single job_posting

In [10]:
def get_jobURL_and_jk(href):
    jk_index = href.find("?jk=")
    if jk_index > 0:
        view_job = "https://ca.indeed.com/viewjob"
        #'/rc/clk?jk=3a3311124a5e8634&fccid=14a67846d05c8406&vjs=3'
        jk_index = href.find("?jk=")
        end_index = href[jk_index:].find("&")
        jk_id = href[jk_index + 4:jk_index + end_index]
        job_link = view_job + "?jk=" + jk_id   
    elif '/company/' in href:
        ca_indeed = "https://ca.indeed.com" 
        #'/company/Yappn-Canada-Inc/jobs/Machine-Learning-Engineer-489466b291064d2a?fccid=7a6bbd753931f130&vjs=3'
        start = href.find('/jobs/')
        end = href[start:].find('?')
        jk_id = href[start:start+end][-16:]
        job_link = ca_indeed + href
    else:
        return None,None
    return job_link,jk_id


#### Function: get_job_search_results(job_name,city_name,province_name)
Input :
- job_name : string containing the job_name used for the search on indeed.com
- city_name : string containing name of the city for the search. Spaces are allowed.
- province_name : string containing name of the province for the search. Spaces are allowed.

Steps included in this function:
- make a request to indeed.com to seaching for a job_name in a particular location
- for every listing on every result page, record the jk_id, url and page number where the listing was found. Store this info in the SQL Job_Search table
- create a list of jk_ids and URLs for all the listings in the result page that are not present in the SQL Listing table

Output:
- job_urls : a list of [jk_id , URL] of the listings that were found on the results page and not yet present in the SQL Listing table 

In [11]:
def get_job_search_results(job_name, city_name, province_name):
    #Crete initial link to start a search based on job_name, city_name and province_name
    job = job_name.replace(" ","+")
    city = city_name.replace(" ","+")
    province = province_name.replace(" ","+")
    starter_link = "https://ca.indeed.com/jobs?q=" + job + "&l=" + city + "%2C+" + province + "&start=0"
    
    #Keep a list that stores all the job URLs that need to be accessed
    #These jobs are not in the listing table
    job_urls = []
    
    #Get the job links
    keep_going = True #determined by the presence of Next button on the page
    page_num = 1
    page_link = starter_link
    while keep_going:
        # Get the html code on the first page that contains multiple job postings
        try:
            r = requests.get(page_link)
        except Exception as e:
            logging.error(f"Requests ERROR: {page_link} could not access the result search page {page_num}.")
            logging.error(e)
            continue
        if r.status_code == 200:
            page_soup = bs(r.text,"html.parser")
            # Get all the listings on the page
            listings = page_soup.find_all('div', class_ = "jobsearch-SerpJobCard")
            # For each listing on the result search page
            for listing in listings:
                href = listing.a["href"]
                job_link,jk_id = get_jobURL_and_jk(href)
                if job_link == None or jk_id == None:
                    logging.info(f"Job_link and jk_id NOT FOUND in href {href}")
                    continue
                #Check to see if the job is already in listings
                if session.query(Listing).filter(Listing.jk_id == jk_id).count() == 0:
                    job_urls.append([jk_id,job_link])
                #Add listing meta data to Job_Search
                job_search = Job_Search(job_search_word = job_name,
                                       city = city_name,
                                       province = province_name,
                                       page = page_num,
                                       jk_id = jk_id,
                                       url = job_link)
                session.add(job_search)
            session.commit()

            #Check to see if there is a next page
            prev_next_buttons = page_soup.find_all('span','np')
            if len(prev_next_buttons) == 0:
                logging.info(f"No Next or Previous buttons were found on page {page_num} when searching {job_name} \
                in {city_name},{province_name}. No span np tag were found.")
                keep_going = False
            else:
                if "Next" in prev_next_buttons[0].text or "Next" in prev_next_buttons[-1].text:
                    page_num += 1
                    eq_index = -page_link[::-1].find("=")
                    page_link = page_link[:eq_index] + str(int(page_link[eq_index:]) + 10)
                else:
                    keep_going = False
                time.sleep(random.randint(1,10))
        #keep_going = False #####
    return job_urls

#### Function: get_job_listings(job_urls)
Input :
- job_urls : a list of lists. Each item in the list is a list with two items: [jk_id, URL]. These are the jk_id and the URL of listings that are not currently present in the listings table.

Steps included in this function:
- Create a folder if it does not exist to store the .html files.
- For each list in job_urls do the following tasks:
    - Access the job listing URL. Get the jk_id of all the recommended jobs on the page. For each recommended job:
        - Add a new row to the recommendation table
        - Check to see if the recommended job is present in listings table. If it is not, make a request to the recommendation URL, save the HTML of the recommendation listing to file, find the recommendations present on this recommendation listing and add a new row to the listings table. 
    - Save the HTML code of the posting to a .html file. 

Output:
- The function does not return an output. However, it creates a Job_Postings_Raw_HTML folder locally if it does not exist and adds to it .html files that contain individual job postings.

In [12]:
# Access all the new jobs found on the results pages and download the html page of the posting. 
# All the listings in job_urls are not present in the Listings table
def get_job_listings(job_urls):
    #Create file for html job posts if it does not exist 
    folder = "Job_Postings_Raw_HTML"
    job_path = os.path.join(folder)
    if not os.path.exists(job_path):
        os.makedirs(job_path)  
    
    # Make a url request, get the html code and store it in an html file
    # Add info to listings table
    for jk_url in job_urls:
        jk_id = jk_url[0]
        url = jk_url[1]
        try:
            r = requests.get(url)
        except Exception as e:
            logging.error(f"Requests ERROR: {jk_url} could not be accessed.")
            logging.error(e)
            continue
        soup = bs(r.text, 'html.parser')
        title = soup.title.string
        
        #Find the recommendations and store them in the file
        recs = soup.find_all('div',class_ = "icl-JobResult")
        rec_jk = [] #List that will store 5 jk IDs 
        rec_url = [] # List that will store 5 URLs
        for i in range(len(recs)):
            if recs[i].find("a") != None:
                href_rec = recs[i].a["href"]
                job_link, job_jk = get_jobURL_and_jk(href_rec)
                rec_jk.append(job_jk)
                rec_url.append(job_link)
                recommendation = Recommendation(jk_id = job_jk, rec_url = job_link, parent_jk = jk_id)
                session.add(recommendation)
        session.commit()
        
        #Check to see if the recommended listings were already scraped, if not, scrape them.
        
        for j in range(len(rec_jk)):
            #if session.query(Listing.jk_id).filter_by(jk_id = rec_jk[j]).scalar() is None:
            if session.query(Listing.jk_id).filter_by(jk_id = rec_jk[j]).first() is None:
                #The recommeded posting was not scraped before
                try:
                    rec_request = requests.get(rec_url[j]).text
                except Exception as e:
                    logging.error(f"Recommendation URL {rec_url[j]} could not be accessed")
                    logging.error(e)
                    continue
                rec_soup = bs(rec_request, 'html.parser')
                rec_title = rec_soup.title.string
                #Write HTML code to file 
                file_name_recommendation = ''.join(e for e in title if e.isalnum() or e == " ") + "_" + rec_jk[j]
                current_path_rec = os.path.join(folder,file_name_recommendation + " RAW.html")
                with open(current_path_rec,'w') as f:
                    f.write(str(rec_soup))
                #Find the jk_id of the recommended jobs on this page.
                recs_on_rec = rec_soup.find_all('div',class_ = "icl-JobResult")
                recs_on_rec_jk = []
                for rec_on_rec in recs_on_rec:
                    if rec_on_rec.find("a") != None:
                        href_rec_on_rec = rec_on_rec.a["href"]
                        rec_on_rec_link, rec_on_rec_jk = get_jobURL_and_jk(href_rec_on_rec)
                        recs_on_rec_jk.append(rec_on_rec_jk)
                for i in range(5-len(recs_on_rec_jk)):
                    recs_on_rec_jk.append(None)
                recommendation_listing = Listing(jk_id = rec_jk[j], title = rec_title, 
                                                 recommendation1_jk = recs_on_rec_jk[0],recommendation2_jk = recs_on_rec_jk[1],
                                                recommendation3_jk = recs_on_rec_jk[2],recommendation4_jk = recs_on_rec_jk[3],
                                                recommendation5_jk = recs_on_rec_jk[4])
                session.add(recommendation_listing)
                #print("Added recommention " + str(rec_jk[j]))
                session.commit()
        
        #If the listing has less than 5 recommendations, add None to the list to make it 5.
        for i in range(5 - len(rec_jk)):
            rec_jk.append(None)
        
        
        listing = Listing(jk_id = jk_id, title = title, recommendation1_jk = rec_jk[0],
                         recommendation2_jk = rec_jk[1], recommendation3_jk = rec_jk[2],
                         recommendation4_jk = rec_jk[3], recommendation5_jk = rec_jk[4])
        session.add(listing)
        session.commit()
        
        file_name = ''.join(e for e in title if e.isalnum() or e == " ") + "_" + jk_id
        current_path = os.path.join(folder,file_name + " RAW.html")
        with open(current_path,'w') as f:
            f.write(str(soup))
        time.sleep(random.randint(1,5))

## Program Starts Here

In [22]:
jobs = ["Machine Learning"]
cities = ["Toronto,ON"]
for job in jobs:
    for city_prov in cities:
        city, province = city_prov.split(",") 
        urls = get_job_search_results(job,city,province)
        get_job_listings(urls)

See rows in the Job_Search table 

In [27]:
job_search = pd.read_sql(session.query(Job_Search).statement,session.bind)
job_search.head()

,id,job_search_word,city,province,page,jk_id,url
0,1,Machine Learning,Toronto,ON,1,0771eaacf4968c0e,https://ca.indeed.com/viewjob?jk=0771eaacf4968c0e
1,2,Machine Learning,Toronto,ON,1,ebb7e6ef37a92b0b,https://ca.indeed.com/viewjob?jk=ebb7e6ef37a92b0b
2,3,Machine Learning,Toronto,ON,1,91b267ed19b7f17f,https://ca.indeed.com/viewjob?jk=91b267ed19b7f17f
3,4,Machine Learning,Toronto,ON,1,06d2ed9ab3906eb2,https://ca.indeed.com/company/Lighthouse-Outco...
4,5,Machine Learning,Toronto,ON,1,8f7a514edfc87312,https://ca.indeed.com/viewjob?jk=8f7a514edfc87312


See rows in the Listings table

In [28]:
listing_df = pd.read_sql(session.query(Listing).statement,session.bind) 
listing_df.head()

,id,jk_id,title,recommendation1_jk,recommendation2_jk,recommendation3_jk,recommendation4_jk,recommendation5_jk
0,1,81e86a6105eb4edf,"Engineering Lead, Machine Learning - Toronto, ...",a1e995a14f4dfa02,fa7d4635d8ac8682,06d2ed9ab3906eb2,e95b867cee97a2c3,27fea6dc5ce62dce
1,2,e95b867cee97a2c3,"Senior Machine Learning Developer - Toronto, O...",4477fb340c8570ed,a1e995a14f4dfa02,38287aa6ea041b89,dde6a10c9a18ca4f,032628047ff0c65b
2,3,a1e995a14f4dfa02,"Senior Machine Learning Engineer - Toronto, ON...",1da64ca0749b4972,81e86a6105eb4edf,fa7d4635d8ac8682,7d48803c5e18b623,06d2ed9ab3906eb2
3,4,565575a8cf0ec8f8,Postdoctoral Fellow in Image Processing and Ma...,8e55568e2a08fcd6,a776f41a9f45d68b,f683841e3723e018,None,None
4,5,06d2ed9ab3906eb2,"Research Associate, Machine Learning - Toronto...",a07f82e44225db65,06cb222dac5a9111,e95b867cee97a2c3,cbbc7e8a7521f8fd,None


See rows in the Recommendation table

In [29]:
recommendation = pd.read_sql(session.query(Recommendation).statement,session.bind) 
recommendation.head()

,id,jk_id,rec_url,parent_jk
0,1,81e86a6105eb4edf,https://ca.indeed.com/viewjob?jk=81e86a6105eb4edf,0771eaacf4968c0e
1,2,e95b867cee97a2c3,https://ca.indeed.com/viewjob?jk=e95b867cee97a2c3,0771eaacf4968c0e
2,3,a1e995a14f4dfa02,https://ca.indeed.com/viewjob?jk=a1e995a14f4dfa02,0771eaacf4968c0e
3,4,565575a8cf0ec8f8,https://ca.indeed.com/viewjob?jk=565575a8cf0ec8f8,0771eaacf4968c0e
4,5,06d2ed9ab3906eb2,https://ca.indeed.com/viewjob?jk=06d2ed9ab3906eb2,0771eaacf4968c0e
